In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy as sc
from scipy.spatial.distance import pdist, squareform
from itertools import combinations
from scipy import stats
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [3]:
def kerval(a, b, p = 3):
    #Polunomial kernel
    return (a.T@b)**p

#check if this is even used
def mysign(x):
    if x <= 0:
        return -1
    else:
        return 1
    
#sign to create array for real y used in the multiclass perceptron
def mysign2(x):
    check = (x<=0)
    n = x.shape[0]
    new_x = np.ones([n])
    new_x[check] = -1
    
    return new_x

# a function to shuffle data randomly
def shuffling(data):
    rng = np.random.default_rng()
    rng.shuffle(data, axis=0)
    return data

# to generate an array of zeros for alpha of perceptron
def clearClass(data,numb_class = 3):
    return np.zeros((numb_class, data.shape[0]))

#a function for generating different folds for cross validation
def cross_data(data,folds):
    folds = int(folds)
    #data = np.loadtxt(path)
    
    rng = np.random.default_rng()

    rng.shuffle(data, axis=0)
    num = int(data.shape[0]/folds)
    folds_Arr = np.zeros([folds,num,int(data.shape[1])])
    #m = data.shape[0]
    bound_prev = 0
    bound_current = 0
    #print("shape",data.shape)
    for a in range(folds):
        bound_current += num
        #print(a)
        #print(bound_prev ,bound_current)
        folds_Arr[a] = data[bound_prev:bound_current]
        bound_prev = bound_current
    
    return folds_Arr

# a function to split data 80 train 20 test
def train_test_split(datas):
    data = datas

    rng = np.random.default_rng()

    rng.shuffle(data, axis=0)
    #print(data.shape)

    m = data.shape[0]

    train = data[0:int(m*0.8)]
    test = data[int(0.8*m):]

    return train, test

# a function to check with the test data and produce error rate
def testClass3(data,test_data,classifiers,numb_class,d,kernel_result):
    mistakes = 0
    preds = np.zeros((numb_class))
    maxi = 0
    for i in range(test_data.shape[0]):
        preds = (classifiers@kernel_result[i].reshape(-1,1)).T[0]
        
        y_pred = np.argmax(preds)
        y_true = int(test_data[i,0])   
        if y_pred != y_true:
            mistakes+=1
    # error rate test sample
    return mistakes/test_data.shape[0]

# a function to check with test data and produce error rate for
# shld change name later on
def testClass31(data,test_data,classifiers,numb_class,d,kernel_result):
    mistakes = 0
    preds = np.zeros((numb_class))
    maxi = 0
    confu = np.zeros([numb_class,numb_class])
    for i in range(test_data.shape[0]):
        preds = (classifiers@kernel_result[i].reshape(-1,1)).T[0]
        
        y_pred = np.argmax(preds)
        y_true = int(test_data[i,0])   
        if y_pred != y_true:
            confu[int(y_pred),int(y_true)] +=1
            mistakes+=1
        #print("mistakes",mistakes,y_pred,y_true)
    return mistakes,confu

#a training function to learn from training set and produce alphas required for
def traingen23(data, classifiers,numb_class , d,kernel_result):
    mistakes = 0
    preds = np.zeros((numb_class))
    maxi = 0
    
    for i in range(data.shape[0]):

     
        preds = (classifiers@kernel_result[i].reshape(-1,1)).T[0]

        #print("preds",preds.shape)

        y_pred = np.argmax(preds)
        y_true = int(data[i,0])

        Y_true = np.ones(numb_class)*-1
        Y_true[y_true] = 1

        y_mistakes = ((preds*Y_true) <= 0)
        #print(y_mistakes.shape)

        classifiers[y_mistakes, i] = classifiers[y_mistakes, i] - mysign2(preds[y_mistakes])  

        if y_pred != y_true:
            mistakes+=1
        
            
    return mistakes/data.shape[0], classifiers



<h3> SVM <h3>

In [4]:
def point2(i1,m):
    i2 = np.random.choice(m)
    while i1 == i2:
        i2 = np.random.choice(m)

    return i2

In [5]:
#latest svm implementation
def SMO(X, y, C,K ):
    tol=10e-2
    max_passes=10
    rand_no_prog = 0
    m = X.shape[0]
    alphas = np.zeros((m))
#     old_alphas = np.zeros((m))
    b = 0
    E = np.ones((m))*(-y)

#     K = (X @ X.T) ** d
    # kernel = lambda xi, yi: (np.dot(xi.T, yi)) ** d

    passes = 0
    time_out = 0
    while (passes < max_passes) and (time_out < 10):
        #print("time",time_out)
        num_changed_alphas = 0
        for i in range(m):
            rand_no_prog = 0
            #print("iterate",i)
            
            E[i] = alphas * y @ K[i, :] + b - y[i]

            # E[i] = np.sum(alphas*y*kernel(X.T, X[i,:])) + b - y[i]

            if (y[i]*E[i] < -tol and alphas[i] < C)\
            or (y[i]*E[i] > tol and alphas[i] > 0): 

                while rand_no_prog < 5:
                    #print(f"progress {rand_no_prog}, iter{i}")

                    j = point2(i,m)



                    E[j] = (alphas * y) @K[j, :] + b - y[j]


                    if y[i] != y[j]:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(C, C + alphas[j] - alphas[i])

                    else:
                        L = max(0, alphas[i] + alphas[j] - C)
                        H = min(C, alphas[i] + alphas[j])

                    if L == H:
                        rand_no_prog +=1
                        continue

                    eta = 2*K[i, j] - K[i, i] - K[j, j]
                    # eta = 2*kernel(X[i,:], X[j,:]) - kernel(X[i,:], X[i,:]) - kernel(X[j,:], X[j,:])

                    if eta >= 0:
                        rand_no_prog +=1
                        continue

                    # Compute alpha_j
                    alphasj = alphas[j] - (y[j]*(E[i] - E[j]) / eta)

                    # Clip alpha_j

                    if alphasj > H:
                        alphasj = H
                    elif alphasj < L:
                        alphasj = L
                    # Note: alpha_j is unchanged if L<=alpha_j<=H as desired

                    if abs(alphasj - alphas[j]) < 10e-100:
                        rand_no_prog +=1
                        #print("no progress")
                        continue

                    alphasi = alphas[i] + (y[i] * y[j] * (alphas[j] - alphasj))

                    b1 = b - E[i] - (y[i]*(alphasi - alphas[i])*K[i,i]) - (y[j]*(alphasj - alphas[j])*K[i,j])
                    # b1 = b - E[i] - y[i]*(alphas[i] - old_alpha_i)*kernel(X[i,:], X[i,:]) - y[j]*(alphas[j] - old_alpha_j)*kernel(X[i,:], X[j,:])

                    b2 = b - E[j] - (y[i]*(alphasi - alphas[i])*K[i,j]) - (y[j]*(alphasj - alphas[j])*K[j,j])
                    
                    alphas[i] = alphasi
                    alphas[j] = alphasj
                    if (alphas[i] > 0) and (alphas[i] < C):
                        b = b1
                    elif (alphas[j] > 0) and (alphas[j] < C):
                        b = b2
                    else:
                        b = (b1 + b2) / 2

                    num_changed_alphas += 1
                    #exit loop to change i1 or first point
                    rand_no_prog = 1000

                    
                    
                    
            
            # print("i error: ", E[i], "j error: ", E[j])
        if num_changed_alphas == 0:
            passes += 1
        else:
            passes = 0
        
        time_out += 1
    
    return alphas, b, E

In [6]:
def train_svm(combi,train,d,c):
    a = []
    betas = np.zeros([45,1])
    time_start = time.time()

    for i,comb in enumerate(combi):

        label0 = comb[0] # +1
        label1 = comb[1] # -1

        picked_1 = (train[:,0] == label1)
        picked_0 = (train[:,0] == label0)


        samples = np.logical_or(picked_1, picked_0)

        index_label1 = np.where(picked_1 == True)[0]
        index_label0 = np.where(picked_0 == True)[0]

        train_1 = train[index_label1]
        train_0 = train[index_label0]
        train_1[:,0] = -1
        train_0[:,0] = 1
        non_index = np.where(samples == False)[0]
        data_train = np.concatenate([train_0, train_1], axis = 0)

        small_x = data_train[:,1:]
        small_y = data_train[:,0]


        small_kernel = kerval(small_x.T, small_x.T, d)

        aresult,betas[i],Eresult = SMO(small_x, small_y,c,small_kernel )
        a.append(aresult)


    print("time taken training", time.time()-time_start)
    return a, betas

In [7]:
def pred_class(pred_vals, combo):

    which_class = 1* (pred_vals <= 0) 
    classifier_preds = combo[np.arange(combo.shape[0]), which_class]
    
#     # Count class votes
#     unique, counts = np.unique(classifier_preds, return_counts = True)
    
#     # Prediction is just maximum counted class
#     pred_lab = unique[np.argmax(counts)]

    pred_lab = stats.mode(classifier_preds)[0][0]
    
    return pred_lab

In [8]:
def preds_data(combi,kernel_test,test,train,a,betas):
    preds_data = np.zeros([test.shape[0],45])
    start_time = time.time()
    for i,a_val in enumerate(a):
        combi_curr = combi[i]
        label0 = combi_curr[0] # +1
        label1 = combi_curr[1] # -1
        #print()
        #print("label",label0,label1)

        picked_1 = (train[:,0] == label1)
        picked_0 = (train[:,0] == label0)
        #print("sum",np.sum(picked_0)+np.sum(picked_1))


#         samples = np.logical_or(picked_1, picked_0)

        index_label1 = np.where(picked_1 == True)[0]
        index_label0 = np.where(picked_0 == True)[0]
        train_1 = train[index_label1]
        train_0 = train[index_label0]
        train_1[:,0] = -1
        train_0[:,0] = 1
        data_label = np.concatenate([index_label0, index_label1], axis = 0)
        small_y = np.concatenate([train_0[:,0], train_1[:,0]], axis = 0)

        kern_relevant = kernel_test[data_label]
        alpha = np.array(a_val)

        pred = (alpha*small_y).reshape(1,-1)@kern_relevant + betas[i]
        preds_data[:,i] = np.sign(pred.reshape(-1))
    print("time taken predicting", time.time() - start_time)
    return preds_data

In [ ]:
# trial code to determine range of c
#power above 1 has no effect
#power below 10-5 produce result that is very bad
cset = np.array([10.0**(-4),10.0**(-3),10.0**(-2), 1])
path_real_data = "./zipcombo.dat"
datas = np.loadtxt(path_real_data)
m,n = datas.shape
iteration = 1
diffDs = np.arange(1,2,1)
error_sum_svm_trial = np.zeros([iteration , len(diffDs) , len(cset) ,2])
cValues = np.array([10**(50),1,10**(-50)])
time_total = time.time()
for iters in range (iteration):
    train,test = train_test_split(datas)
    time_inter = time.time()
    for dindex, d in enumerate(diffDs):
        for cindex, c in enumerate(cset):
            print(f"d {d}, c {c}, iter {iters}")
            time_d = time.time()
            combi = list (combinations(range(10),2))

            a, betas = train_svm(combi,train,d,c)

            kernel_test = kerval(train[:,1:].T, test[:,1:].T, d)
            pred_result = preds_data(combi,kernel_test,test,train,a,betas)

            pred_final = np.zeros([test.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))

            avg_test_err = np.mean(pred_final != test[:,0])
            kernel_train = kerval(train[:,1:].T, train[:,1:].T, d)
            pred_result = preds_data(combi,kernel_train,train,train,a,betas)

            pred_final = np.zeros([train.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))
            avg_train_err = np.mean(pred_final != train[:,0])
            error_sum_svm_trial[iters,dindex,cindex] = np.array([avg_train_err,avg_test_err])
            print(f"print train and test error {avg_train_err} and {avg_test_err} ")
            print(f"time to run {d} d",time.time() - time_d)
    print(f"iteration {iters}",time.time()-time_inter)
print("total time", time.time()-time_total)

In [13]:
# trial code to determine range of c
#power above 1 has no effect
#power below 10-5 produce result that is very bad
cset = np.array([10.0**(-3),10.0**(-2), 1])
path_real_data = "./zipcombo.dat"
datas = np.loadtxt(path_real_data)
m,n = datas.shape
iteration = 1
diffDs = np.arange(1,2,1)
error_sum_svm_trial = np.zeros([iteration , len(diffDs) , len(cset) ,2])
time_total = time.time()
for iters in range (iteration):
    train,test = train_test_split(datas)
    time_inter = time.time()
    for dindex, d in enumerate(diffDs):
        for cindex, c in enumerate(cset):
            print(f"d {d}, c {c}, iter {iters}")
            time_d = time.time()
            combi = list (combinations(range(10),2))

            a, betas = train_svm(combi,train,d,c)

            kernel_test = kerval(train[:,1:].T, test[:,1:].T, d)
            pred_result = preds_data(combi,kernel_test,test,train,a,betas)

            pred_final = np.zeros([test.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))

            avg_test_err = np.mean(pred_final != test[:,0])
            kernel_train = kerval(train[:,1:].T, train[:,1:].T, d)
            pred_result = preds_data(combi,kernel_train,train,train,a,betas)

            pred_final = np.zeros([train.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))
            avg_train_err = np.mean(pred_final != train[:,0])
            error_sum_svm_trial[iters,dindex,cindex] = np.array([avg_train_err,avg_test_err])
            print(f"print train and test error {avg_train_err} and {avg_test_err} ")
            print(f"time to run {d} d",time.time() - time_d)
    print(f"iteration {iters}",time.time()-time_inter)
print("total time", time.time()-time_total)

d 1, c 0.001, iter 0
time taken training 14.491607427597046
time taken predicting 0.4802680015563965
time taken predicting 1.6069791316986084
print train and test error 0.05229900510890024 and 0.060215053763440864 
time to run 1 d 18.51906180381775
d 1, c 0.01, iter 0
time taken training 14.928883790969849
time taken predicting 0.45079636573791504
time taken predicting 1.681509256362915
print train and test error 0.026888948642108095 and 0.05 
time to run 1 d 19.003029346466064
d 1, c 1.0, iter 0
time taken training 16.298166275024414
time taken predicting 0.49165940284729004
time taken predicting 1.6238343715667725
print train and test error 0.028098951331002958 and 0.06344086021505377 
time to run 1 d 20.404479026794434
iteration 0 57.92657017707825
total time 57.94450235366821


In [14]:
# main run
#power above 1 has no effect
#power below 10-5 produce result that is very bad
cset = np.array([10.0**(-3),10.0**(-2), 1])
path_real_data = "./zipcombo.dat"
datas = np.loadtxt(path_real_data)
m,n = datas.shape
iteration = 20
diffDs = np.arange(1,8,1)
error_sum_svm_trial = np.zeros([iteration , len(diffDs) , len(cset) ,2])

time_total = time.time()
for iters in range (iteration):
    train,test = train_test_split(datas)
    time_inter = time.time()
    for dindex, d in enumerate(diffDs):
        for cindex, c in enumerate(cset):
            print(f"d {d}, c {c}, iter {iters}")
            time_d = time.time()
            combi = list (combinations(range(10),2))

            a, betas = train_svm(combi,train,d,c)

            kernel_test = kerval(train[:,1:].T, test[:,1:].T, d)
            pred_result = preds_data(combi,kernel_test,test,train,a,betas)

            pred_final = np.zeros([test.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))

            avg_test_err = np.mean(pred_final != test[:,0])
            kernel_train = kerval(train[:,1:].T, train[:,1:].T, d)
            pred_result = preds_data(combi,kernel_train,train,train,a,betas)

            pred_final = np.zeros([train.shape[0]])
            for i,pred_data in enumerate(pred_result):
                pred_final[i] = pred_class(pred_data, np.array(combi))
            avg_train_err = np.mean(pred_final != train[:,0])
            error_sum_svm_trial[iters,dindex,cindex] = np.array([avg_train_err,avg_test_err])
            print(f"print train and test error {avg_train_err} and {avg_test_err} ")
            print(f"time to run {d} d",time.time() - time_d)
    print(f"iteration {iters}",time.time()-time_inter)
print("total time", time.time()-time_total)

d 1, c 0.001, iter 0
time taken training 15.56591510772705
time taken predicting 0.5136313438415527
time taken predicting 1.8480870723724365
print train and test error 0.049475665501478895 and 0.06827956989247312 
time to run 1 d 20.06690740585327
d 1, c 0.01, iter 0
time taken training 16.228090047836304
time taken predicting 0.4906904697418213
time taken predicting 1.699038028717041
print train and test error 0.028098951331002958 and 0.054838709677419356 
time to run 1 d 20.4732985496521
d 1, c 1.0, iter 0
time taken training 16.31690740585327
time taken predicting 0.47300100326538086
time taken predicting 1.6964428424835205
print train and test error 0.033476741059424574 and 0.07688172043010753 
time to run 1 d 20.531322240829468
d 2, c 0.001, iter 0
time taken training 16.056121110916138
time taken predicting 0.5156266689300537
time taken predicting 1.7134246826171875
print train and test error 0.0010755579456843238 and 0.035483870967741936 
time to run 2 d 20.39606547355652
d 2, c

time taken training 19.178788900375366
time taken predicting 0.476729154586792
time taken predicting 1.6904776096343994
print train and test error 0.00026888948642108095 and 0.024731182795698924 
time to run 5 d 25.92455554008484
d 5, c 0.01, iter 1
time taken training 19.13341212272644
time taken predicting 0.48369550704956055
time taken predicting 1.7083897590637207
print train and test error 0.00013444474321054048 and 0.020967741935483872 
time to run 5 d 25.897305250167847
d 5, c 1.0, iter 1
time taken training 20.498961448669434
time taken predicting 0.5005841255187988
time taken predicting 1.6864662170410156
print train and test error 0.00040333422963162143 and 0.021505376344086023 
time to run 5 d 27.231428384780884
d 6, c 0.001, iter 1
time taken training 18.99177360534668
time taken predicting 0.47274255752563477
time taken predicting 1.7054457664489746
print train and test error 0.00026888948642108095 and 0.024731182795698924 
time to run 6 d 25.752746105194092
d 6, c 0.01, i

time taken predicting 1.7173876762390137
print train and test error 0.0008066684592632429 and 0.02849462365591398 
time to run 2 d 19.438494443893433
d 2, c 0.01, iter 3
time taken training 15.143767833709717
time taken predicting 0.5018091201782227
time taken predicting 1.799274206161499
print train and test error 0.0 and 0.030107526881720432 
time to run 2 d 19.52729344367981
d 2, c 1.0, iter 3
time taken training 15.219273567199707
time taken predicting 0.4837150573730469
time taken predicting 1.6627376079559326
print train and test error 0.00040333422963162143 and 0.032795698924731186 
time to run 2 d 19.364097118377686
d 3, c 0.001, iter 3
time taken training 18.536332607269287
time taken predicting 0.4417698383331299
time taken predicting 1.6815013885498047
print train and test error 0.00013444474321054048 and 0.027419354838709678 
time to run 3 d 24.820382595062256
d 3, c 0.01, iter 3
time taken training 18.541146516799927
time taken predicting 0.4637331962585449
time taken pred

time taken training 18.130388259887695
time taken predicting 0.4517648220062256
time taken predicting 1.6760704517364502
print train and test error 0.00026888948642108095 and 0.026881720430107527 
time to run 6 d 24.42252540588379
d 6, c 1.0, iter 4
time taken training 18.093058109283447
time taken predicting 0.45174312591552734
time taken predicting 1.8346295356750488
print train and test error 0.00026888948642108095 and 0.025806451612903226 
time to run 6 d 24.86864423751831
d 7, c 0.001, iter 4
time taken training 21.062158584594727
time taken predicting 0.45973968505859375
time taken predicting 1.8777408599853516
print train and test error 0.00013444474321054048 and 0.03494623655913978 
time to run 7 d 27.93925929069519
d 7, c 0.01, iter 4
time taken training 20.44521427154541
time taken predicting 0.4417724609375
time taken predicting 2.2232723236083984
print train and test error 0.00013444474321054048 and 0.032795698924731186 
time to run 7 d 27.868155241012573
d 7, c 1.0, iter 4

time taken training 18.55628228187561
time taken predicting 0.4820382595062256
time taken predicting 1.6954941749572754
print train and test error 0.00013444474321054048 and 0.021505376344086023 
time to run 3 d 24.87708067893982
d 3, c 1.0, iter 6
time taken training 18.608316898345947
time taken predicting 0.49025893211364746
time taken predicting 1.7163057327270508
print train and test error 0.00026888948642108095 and 0.02258064516129032 
time to run 3 d 25.092225790023804
d 4, c 0.001, iter 6
time taken training 18.857666492462158
time taken predicting 0.4734079837799072
time taken predicting 1.7368824481964111
print train and test error 0.00013444474321054048 and 0.02043010752688172 
time to run 4 d 25.22893500328064
d 4, c 0.01, iter 6
time taken training 18.62702775001526
time taken predicting 0.46581292152404785
time taken predicting 1.7104344367980957
print train and test error 0.00040333422963162143 and 0.01935483870967742 
time to run 4 d 24.969865560531616
d 4, c 1.0, iter 

...

time taken training 18.103947162628174
time taken predicting 0.4796733856201172
time taken predicting 1.712632656097412
print train and test error 0.00026888948642108095 and 0.025806451612903226 
time to run 5 d 24.511027574539185
d 5, c 1.0, iter 12
time taken training 18.185750484466553
time taken predicting 0.46872711181640625
time taken predicting 1.7281112670898438
print train and test error 0.00013444474321054048 and 0.026344086021505377 
time to run 5 d 24.509482860565186
d 6, c 0.001, iter 12
time taken training 18.116235971450806
time taken predicting 0.4946441650390625
time taken predicting 1.7737174034118652
print train and test error 0.00013444474321054048 and 0.030107526881720432 
time to run 6 d 24.532070875167847
d 6, c 0.01, iter 12
time taken training 18.19668221473694
time taken predicting 0.47869086265563965
time taken predicting 1.8235251903533936
print train and test error 0.00026888948642108095 and 0.02903225806451613 
time to run 6 d 24.637626886367798
d 6, c 1.0

print train and test error 0.0006722237160527024 and 0.02849462365591398 
time to run 2 d 19.845516204833984
d 2, c 0.01, iter 14
time taken training 15.510436534881592
time taken predicting 0.4803955554962158
time taken predicting 1.788740634918213
print train and test error 0.004840010755579457 and 0.03602150537634408 
time to run 2 d 19.793846607208252
d 2, c 1.0, iter 14
time taken training 15.472883939743042
time taken predicting 0.48669934272766113
time taken predicting 1.761929988861084
print train and test error 0.0010755579456843238 and 0.03118279569892473 
time to run 2 d 19.715150594711304
d 3, c 0.001, iter 14
time taken training 18.763316869735718
time taken predicting 0.47669529914855957
time taken predicting 1.763289213180542
print train and test error 0.006050013444474321 and 0.03225806451612903 
time to run 3 d 25.18615961074829
d 3, c 0.01, iter 14
time taken training 18.953110456466675
time taken predicting 0.521564245223999
time taken predicting 1.664865255355835
pr

time taken training 18.04671549797058
time taken predicting 0.4573178291320801
time taken predicting 1.7470769882202148
print train and test error 0.0012100026888948643 and 0.01935483870967742 
time to run 6 d 24.50261425971985
d 6, c 0.01, iter 15
time taken training 18.09576964378357
time taken predicting 0.47370243072509766
time taken predicting 1.720468521118164
print train and test error 0.00013444474321054048 and 0.01881720430107527 
time to run 6 d 24.485492944717407
d 6, c 1.0, iter 15
time taken training 18.031831741333008
time taken predicting 0.46172356605529785
time taken predicting 1.714076280593872
print train and test error 0.00013444474321054048 and 0.01989247311827957 
time to run 6 d 24.49308133125305
d 7, c 0.001, iter 15
time taken training 17.980788230895996
time taken predicting 0.4876976013183594
time taken predicting 1.6864502429962158
print train and test error 0.00013444474321054048 and 0.024731182795698924 
time to run 7 d 24.317384958267212
d 7, c 0.01, iter

print train and test error 0.0021511158913686476 and 0.030107526881720432 
time to run 2 d 19.516854524612427
d 3, c 0.001, iter 17
time taken training 18.65263533592224
time taken predicting 0.48070764541625977
time taken predicting 1.7415757179260254
print train and test error 0.0005377789728421619 and 0.025806451612903226 
time to run 3 d 25.050703287124634
d 3, c 0.01, iter 17
time taken training 18.509333610534668
time taken predicting 0.45728540420532227
time taken predicting 1.7692759037017822
print train and test error 0.00026888948642108095 and 0.025806451612903226 
time to run 3 d 25.03001117706299
d 3, c 1.0, iter 17
time taken training 18.6117742061615
time taken predicting 0.48485827445983887
time taken predicting 1.75998854637146
print train and test error 0.00013444474321054048 and 0.024193548387096774 
time to run 3 d 25.010164737701416
d 4, c 0.001, iter 17
time taken training 18.213345527648926
time taken predicting 0.5046548843383789
time taken predicting 1.747358798

time taken training 18.39611029624939
time taken predicting 0.47569775581359863
time taken predicting 1.7293791770935059
print train and test error 0.00013444474321054048 and 0.023118279569892472 
time to run 6 d 24.855365991592407
d 7, c 0.001, iter 18
time taken training 18.079051971435547
time taken predicting 0.48368048667907715
time taken predicting 1.7760233879089355
print train and test error 0.00013444474321054048 and 0.025268817204301075 
time to run 7 d 24.506686687469482
d 7, c 0.01, iter 18
time taken training 18.426549434661865
time taken predicting 0.5106329917907715
time taken predicting 1.8288872241973877
print train and test error 0.00013444474321054048 and 0.024193548387096774 
time to run 7 d 25.020619869232178
d 7, c 1.0, iter 18
time taken training 18.28012704849243
time taken predicting 0.4996778964996338
time taken predicting 1.7580821514129639
print train and test error 0.00026888948642108095 and 0.024731182795698924 
time to run 7 d 24.755078315734863
iteration

In [16]:

#result processing
for dIndex, resultd in enumerate(diffDs):
    print()
    
    for cIndex,c in enumerate(cset):
        print("d = ",resultd, "c ", cset[cIndex])
        print("mean train",np.mean(error_sum_svm_trial[:,dIndex,cIndex,0]), "standard div train",np.std(error_sum_svm_trial[:,dIndex,cIndex,0]))
        print("mean test",np.mean(error_sum_svm_trial[:,dIndex,cIndex,1]), "standard div test",np.std(error_sum_svm_trial[:,dIndex,cIndex,1]))


d =  1 c  0.001
mean train 0.05239983866630815 standard div train 0.0012039187072178203
mean test 0.057016129032258056 standard div test 0.0049288937425413255
d =  1 c  0.01
mean train 0.028784619521376714 standard div train 0.003191441826920948
mean test 0.045672043010752686 standard div test 0.0045824463872363
d =  1 c  1.0
mean train 0.025914224253831674 standard div train 0.008237182115220338
mean test 0.06220430107526881 standard div test 0.01106286496138729

d =  2 c  0.001
mean train 0.0009343909653132566 standard div train 0.0009348261167527369
mean test 0.02849462365591398 standard div test 0.0038545050169752513
d =  2 c  0.01
mean train 0.0016603925786501748 standard div train 0.0017218771342393641
mean test 0.02849462365591398 standard div test 0.003185228654537338
d =  2 c  1.0
mean train 0.0012100026888948643 standard div train 0.0005767036996377311
mean test 0.029704301075268818 standard div test 0.003704901866163304

d =  3 c  0.001
mean train 0.0010285022855606345 stan

In [23]:
# Cross validation
# 5 folds cross validation
ori_train,ori_test = train_test_split(datas)
num_fold = 5
testTrain = cross_data(ori_train,num_fold)
num_iter = 20
cset = np.array([10.0**(-3),10.0**(-2), 1])
dset = np.arange(1,8,1)
bestResultSvm = np.ones([num_iter,3])
start_time_fold = time.time()
for b in range(num_iter):
    testTrain = cross_data(ori_train,num_fold)
    start_time_iter = time.time()
    for dindex, d in enumerate(dset):
        start_time_main = time.time()

        
        #train_err_rate_new = np.zeros([5,2])
        
        for cindex, c in enumerate(cset):
            errorRateTest = 0
           
            for fold in range(5):

                testFold = testTrain[fold]
                trainFold = np.delete(testTrain,fold,0)
                trainFold = trainFold.reshape(-1,257)

                

                train = trainFold
                test = testFold

                combi = list (combinations(range(10),2))
                #amin loop to predict using One vs one
                a, betas = train_svm(combi,train,d,c)

                kernel_test = kerval(train[:,1:].T, test[:,1:].T, d)
                pred_result = preds_data(combi,kernel_test,test,train,a,betas)

                pred_final = np.zeros([test.shape[0]])
                for i,pred_data in enumerate(pred_result):
                    pred_final[i] = pred_class(pred_data, np.array(combi))

                avg_test_err = np.mean(pred_final != test[:,0])
                kernel_train = kerval(train[:,1:].T, train[:,1:].T, d)
                pred_result = preds_data(combi,kernel_train,train,train,a,betas)

                pred_final = np.zeros([train.shape[0]])
                for i,pred_data in enumerate(pred_result):
                    pred_final[i] = pred_class(pred_data, np.array(combi))
                avg_train_err = np.mean(pred_final != train[:,0])
                #end of one vs one prediction
                #store value in array


                errorRateTest +=avg_test_err
                print("results ",avg_train_err, "mistakes in test",avg_test_err,"for c", c, "for d",d,"for fold ",fold,"iter ",b)
        
        

                
            avgTestErr = errorRateTest/5
            if (avgTestErr < bestResultSvm[b,2]):
                bestResultSvm[b,:] = np.array([d, c, avgTestErr])
        print("--- %s seconds ---" , (time.time() - start_time), "time for d ",d)
            
        
        
    print("--- %s seconds ---" , (time.time() - start_time_iter), "time for iter ",b)
    
print("--- %s seconds ---" , (time.time() - start_time_fold))

time taken training 10.506090879440308
time taken predicting 0.3251347541809082
time taken predicting 1.1100807189941406
results  0.05346334902488231 mistakes in test 0.05917955615332885 for c 0.001 for d 1 for fold  0 iter  0
time taken training 10.371534585952759
time taken predicting 0.3161609172821045
time taken predicting 1.1119976043701172
results  0.05514458641560188 mistakes in test 0.07128446536650974 for c 0.001 for d 1 for fold  1 iter  0
time taken training 10.60810375213623
time taken predicting 0.3122100830078125
time taken predicting 1.1349670886993408
results  0.054640215198386015 mistakes in test 0.05110961667787491 for c 0.001 for d 1 for fold  2 iter  0
time taken training 10.694396495819092
time taken predicting 0.33065295219421387
time taken predicting 1.1200430393218994
results  0.049932750504371216 mistakes in test 0.05783456624075319 for c 0.001 for d 1 for fold  3 iter  0
time taken training 10.347329378128052
time taken predicting 0.33510804176330566
time take

time taken training 13.785548448562622
time taken predicting 0.3331108093261719
time taken predicting 1.1010627746582031
results  0.0018493611297915266 mistakes in test 0.02757229320780094 for c 0.01 for d 3 for fold  1 iter  0
time taken training 13.903995037078857
time taken predicting 0.2961759567260742
time taken predicting 1.0582008361816406
results  0.0003362474781439139 mistakes in test 0.01546738399462004 for c 0.01 for d 3 for fold  2 iter  0
time taken training 14.021440505981445
time taken predicting 0.32114219665527344
time taken predicting 1.090087890625
results  0.0253866845998655 mistakes in test 0.0410221923335575 for c 0.01 for d 3 for fold  3 iter  0
time taken training 13.973357677459717
time taken predicting 0.33211326599121094
time taken predicting 1.1020252704620361
results  0.0006724949562878278 mistakes in test 0.025554808338937456 for c 0.01 for d 3 for fold  4 iter  0
time taken training 13.816987037658691
time taken predicting 0.29920315742492676
time taken p

time taken training 13.817901134490967
time taken predicting 0.3061809539794922
time taken predicting 1.1169848442077637
results  0.0003362474781439139 mistakes in test 0.014122394082044385 for c 1.0 for d 5 for fold  2 iter  0
time taken training 13.82383131980896
time taken predicting 0.3112058639526367
time taken predicting 1.1350481510162354
results  0.0003362474781439139 mistakes in test 0.03026227303295225 for c 1.0 for d 5 for fold  3 iter  0
time taken training 13.818774938583374
time taken predicting 0.3361055850982666
time taken predicting 1.1190128326416016
results  0.00016812373907195696 mistakes in test 0.02757229320780094 for c 1.0 for d 5 for fold  4 iter  0
--- %s seconds --- 2368.1752767562866 time for d  5
time taken training 13.614493608474731
time taken predicting 0.34807276725769043
time taken predicting 1.1100003719329834
results  0.0 mistakes in test 0.03227975790181574 for c 0.001 for d 6 for fold  0 iter  0
time taken training 13.987467288970947
time taken pred

results  0.05010087424344317 mistakes in test 0.06859448554135844 for c 0.001 for d 1 for fold  2 iter  1
time taken training 10.564526081085205
time taken predicting 0.3201711177825928
time taken predicting 1.095081090927124
results  0.054640215198386015 mistakes in test 0.05043712172158709 for c 0.001 for d 1 for fold  3 iter  1
time taken training 10.684629440307617
time taken predicting 0.31017136573791504
time taken predicting 1.0801143646240234
results  0.055648957632817755 mistakes in test 0.06119704102219233 for c 0.001 for d 1 for fold  4 iter  1
time taken training 11.495258808135986
time taken predicting 0.3481459617614746
time taken predicting 1.0701093673706055
results  0.02639542703429724 mistakes in test 0.05716207128446537 for c 0.01 for d 1 for fold  0 iter  1
time taken training 11.451155424118042
time taken predicting 0.31516218185424805
time taken predicting 1.1336665153503418
results  0.031102891728312036 mistakes in test 0.0363147276395427 for c 0.01 for d 1 for f

time taken predicting 0.29620838165283203
time taken predicting 1.0531857013702393
results  0.0005043712172158709 mistakes in test 0.01882985877605918 for c 0.01 for d 3 for fold  3 iter  1
time taken training 14.022071838378906
time taken predicting 0.2932157516479492
time taken predicting 1.0811107158660889
results  0.0005043712172158709 mistakes in test 0.02757229320780094 for c 0.01 for d 3 for fold  4 iter  1
time taken training 14.043010711669922
time taken predicting 0.29018497467041016
time taken predicting 1.1120319366455078
results  0.00016812373907195696 mistakes in test 0.03295225285810356 for c 1.0 for d 3 for fold  0 iter  1
time taken training 13.988102436065674
time taken predicting 0.3131239414215088
time taken predicting 1.0780785083770752
results  0.018997982515131136 mistakes in test 0.04371217215870881 for c 1.0 for d 3 for fold  1 iter  1
time taken training 13.636099815368652
time taken predicting 0.2782566547393799
time taken predicting 1.0950415134429932
result

...

time taken training 13.42879343032837
time taken predicting 0.27326464653015137
time taken predicting 1.0162794589996338
results  0.0008406186953597848 mistakes in test 0.023537323470073975 for c 0.01 for d 3 for fold  3 iter  18
time taken training 13.471428155899048
time taken predicting 0.27821969985961914
time taken predicting 1.0235838890075684
results  0.0006724949562878278 mistakes in test 0.025554808338937456 for c 0.01 for d 3 for fold  4 iter  18
time taken training 13.464576244354248
time taken predicting 0.290205717086792
time taken predicting 0.9983077049255371
results  0.0003362474781439139 mistakes in test 0.026227303295225286 for c 1.0 for d 3 for fold  0 iter  18
time taken training 13.629791259765625
time taken predicting 0.2652881145477295
time taken predicting 1.0250043869018555
results  0.0003362474781439139 mistakes in test 0.026227303295225286 for c 1.0 for d 3 for fold  1 iter  18
time taken training 13.406341314315796
time taken predicting 0.28221964836120605
t

time taken training 13.17708444595337
time taken predicting 0.2712419033050537
time taken predicting 0.9930522441864014
results  0.00016812373907195696 mistakes in test 0.020174848688634835 for c 1.0 for d 5 for fold  4 iter  18
--- %s seconds --- 34148.15694856644 time for d  5
time taken training 12.980486392974854
time taken predicting 0.2981710433959961
time taken predicting 1.0042777061462402
results  0.0 mistakes in test 0.03160726294552791 for c 0.001 for d 6 for fold  0 iter  18
time taken training 13.027069091796875
time taken predicting 0.2629852294921875
time taken predicting 1.0783452987670898
results  0.0003362474781439139 mistakes in test 0.029589778076664425 for c 0.001 for d 6 for fold  1 iter  18
time taken training 13.104018449783325
time taken predicting 0.2682528495788574
time taken predicting 1.0103018283843994
results  0.00016812373907195696 mistakes in test 0.02488231338264963 for c 0.001 for d 6 for fold  2 iter  18
time taken training 13.08192753791809
time tak

time taken predicting 1.029252529144287
results  0.05262273032952253 mistakes in test 0.06254203093476798 for c 0.001 for d 1 for fold  4 iter  19
time taken training 10.745651483535767
time taken predicting 0.26624250411987305
time taken predicting 1.0262932777404785
results  0.030094149293880296 mistakes in test 0.04909213180901143 for c 0.01 for d 1 for fold  0 iter  19
time taken training 10.837754011154175
time taken predicting 0.277346134185791
time taken predicting 1.013251781463623
results  0.03026227303295225 mistakes in test 0.05110961667787491 for c 0.01 for d 1 for fold  1 iter  19
time taken training 11.03667664527893
time taken predicting 0.29620790481567383
time taken predicting 0.9746019840240479
results  0.026731674512441156 mistakes in test 0.04371217215870881 for c 0.01 for d 1 for fold  2 iter  19
time taken training 10.687830924987793
time taken predicting 0.29222726821899414
time taken predicting 1.015984058380127
results  0.030766644250168125 mistakes in test 0.0

time taken training 13.18009352684021
time taken predicting 0.3001680374145508
time taken predicting 0.9594368934631348
results  0.0 mistakes in test 0.02151983860121049 for c 1.0 for d 3 for fold  0 iter  19
time taken training 13.459378004074097
time taken predicting 0.28321123123168945
time taken predicting 0.9804825782775879
results  0.00016812373907195696 mistakes in test 0.02488231338264963 for c 1.0 for d 3 for fold  1 iter  19
time taken training 13.340631008148193
time taken predicting 0.2852439880371094
time taken predicting 0.9756224155426025
results  0.00016812373907195696 mistakes in test 0.025554808338937456 for c 1.0 for d 3 for fold  2 iter  19
time taken training 13.336869955062866
time taken predicting 0.2952120304107666
time taken predicting 0.9784257411956787
results  0.0003362474781439139 mistakes in test 0.01882985877605918 for c 1.0 for d 3 for fold  3 iter  19
time taken training 13.474771499633789
time taken predicting 0.266324520111084
time taken predicting 0.

results  0.0 mistakes in test 0.02824478816408877 for c 0.001 for d 6 for fold  0 iter  19
time taken training 12.949446201324463
time taken predicting 0.2872002124786377
time taken predicting 1.027256965637207
results  0.0003362474781439139 mistakes in test 0.0242098184263618 for c 0.001 for d 6 for fold  1 iter  19
time taken training 13.074267625808716
time taken predicting 0.3031888008117676
time taken predicting 0.9843757152557373
results  0.00016812373907195696 mistakes in test 0.03026227303295225 for c 0.001 for d 6 for fold  2 iter  19
time taken training 13.049330472946167
time taken predicting 0.27224135398864746
time taken predicting 0.957444429397583
results  0.00016812373907195696 mistakes in test 0.02488231338264963 for c 0.001 for d 6 for fold  3 iter  19
time taken training 13.023967504501343
time taken predicting 0.2872035503387451
time taken predicting 0.969191312789917
results  0.0003362474781439139 mistakes in test 0.02824478816408877 for c 0.001 for d 6 for fold  4

In [28]:
print(bestResultSvm)

[[3.00000000e+00 1.00000000e-03 2.55548083e-02]
 [4.00000000e+00 1.00000000e+00 2.39408204e-02]
 [3.00000000e+00 1.00000000e-02 2.40753194e-02]
 [4.00000000e+00 1.00000000e+00 2.44788164e-02]
 [4.00000000e+00 1.00000000e-02 2.43443174e-02]
 [4.00000000e+00 1.00000000e-02 2.50168124e-02]
 [4.00000000e+00 1.00000000e+00 2.35373235e-02]
 [4.00000000e+00 1.00000000e-03 2.19233356e-02]
 [4.00000000e+00 1.00000000e-03 2.29993275e-02]
 [4.00000000e+00 1.00000000e-02 2.38063215e-02]
 [4.00000000e+00 1.00000000e-02 2.27303295e-02]
 [4.00000000e+00 1.00000000e-03 2.42098184e-02]
 [4.00000000e+00 1.00000000e+00 2.32683255e-02]
 [4.00000000e+00 1.00000000e-02 2.43443174e-02]
 [4.00000000e+00 1.00000000e+00 2.42098184e-02]
 [4.00000000e+00 1.00000000e-02 2.34028245e-02]
 [4.00000000e+00 1.00000000e-02 2.43443174e-02]
 [4.00000000e+00 1.00000000e-03 2.39408204e-02]
 [4.00000000e+00 1.00000000e-03 2.20578346e-02]
 [3.00000000e+00 1.00000000e+00 2.25958305e-02]]


In [25]:
dbest = bestResultSvm[:,0].reshape(-1)
cbest = bestResultSvm[:,1].reshape(-1)
start_time_main = time.time()
#ori_train,ori_test = read_data(datas)

resultsBestCrossSvm = np.zeros([len(dbest),2])
for diter, d in enumerate(dbest):
    #print(a)
    # classifiers = clearGLBcls(train,10)
    #print(classifiers.shape)
    c = cbest[diter]
    
    train = shuffling(ori_train)
    test = shuffling(ori_test)

    start_time = time.time()
    #amin loop to predict using One vs one
    a, betas = train_svm(combi,train,d, c)

    kernel_test = kerval(train[:,1:].T, test[:,1:].T, d)
    pred_result = preds_data(combi,kernel_test,test,train,a,betas)

    pred_final = np.zeros([test.shape[0]])
    for i,pred_data in enumerate(pred_result):
        pred_final[i] = pred_class(pred_data, np.array(combi))

    avg_test_err = np.mean(pred_final != test[:,0])
    kernel_train = kerval(train[:,1:].T, train[:,1:].T, d)
    pred_result = preds_data(combi,kernel_train,train,train,a,betas)

    pred_final = np.zeros([train.shape[0]])
    for i,pred_data in enumerate(pred_result):
        pred_final[i] = pred_class(pred_data, np.array(combi))
    avg_train_err = np.mean(pred_final != train[:,0])
    #end of one vs one prediction
    print("results ",avg_train_err, "mistakes in test",avg_test_err,"for c", c, "for d",d)


    

    #print("--- %s seconds ---" , (time.time() - start_time_epoch), "iter",b)
    resultsBestCrossSvm[diter,:] = np.array([avg_train_err,avg_test_err])
    print("--- %s seconds ---" , (time.time() - start_time), "for d*",diter)
print("--- %s seconds ---" , (time.time() - start_time_main))

time taken training 19.583230018615723
time taken predicting 0.47922730445861816
time taken predicting 1.6452116966247559
results  0.0014788921753159452 mistakes in test 0.02849462365591398 for c 0.001 for d 3.0
--- %s seconds --- 25.97563672065735 for d* 0
time taken training 18.579772233963013
time taken predicting 0.461775541305542
time taken predicting 1.6762514114379883
results  0.00040333422963162143 mistakes in test 0.02204301075268817 for c 1.0 for d 4.0
--- %s seconds --- 24.85861849784851 for d* 1
time taken training 18.666461944580078
time taken predicting 0.44585275650024414
time taken predicting 1.6800816059112549
results  0.0017477816617370262 mistakes in test 0.025806451612903226 for c 0.01 for d 3.0
--- %s seconds --- 24.955713033676147 for d* 2
time taken training 18.666316270828247
time taken predicting 0.5545146465301514
time taken predicting 1.7663295269012451
results  0.00040333422963162143 mistakes in test 0.023118279569892472 for c 1.0 for d 4.0
--- %s seconds --

In [26]:
dbestavg = np.mean(bestResultSvm[:,0])
dbeststd = np.std(bestResultSvm[:,0])

cbestavg = np.mean(bestResultSvm[:,1])
cbeststd = np.std(bestResultSvm[:,1])

errorTestAvg = np.mean(resultsBestCrossSvm[:,1])
errorStdAvg = np.std(resultsBestCrossSvm[:,1])
print(f"d* average {dbestavg} with std deviation {dbeststd}")

print(f"c* average {cbestavg} with std deviation {cbeststd}")
print(f"error test average {errorTestAvg} with std deviation {errorStdAvg}")

d* average 3.85 with std deviation 0.3570714214271425
c* average 0.3043 with std deviation 0.4554578026557455
error test average 0.02424731182795699 with std deviation 0.00453943041458316


In [40]:
print(resultsBestCrossSvm)

[[0.00026889 0.01827957]
 [0.00026889 0.01989247]]
